In [16]:
from lavis.models import load_model_and_preprocess, model_zoo
import torch
from PIL import Image

device = "cuda:2"

# print(model_zoo)

# img_path = (
#     "/home/choi/BrainDecoder/dataset/imageNet_images/n02106662/n02106662_10232.JPEG"
# )
# pil_img = Image.open(img_path)

print(load_model_and_preprocess.)

blip_model, vis_processors, _ = load_model_and_preprocess(
    name="blip_caption",
    model_type="base_coco",
    is_eval=True,
    device=device,
)
pil_img = vis_processors["eval"](pil_img).unsqueeze(0).to(device)
blip_model.generate({"image": pil_image})

<method-wrapper '__init__' of function object at 0x7f2cb8aba700>


TypeError: expected str, bytes or os.PathLike object, not NoneType